# Special kursus

## Import of modules

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]=f'{0}'
from __future__ import annotations

import torch

from modules import dataHandler, dataprocessing, models
from modules import scalers as scaling

%matplotlib inline
%load_ext autoreload
%autoreload 2

/zhome/ae/4/138518/Desktop/env/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"cuda {torch.cuda.current_device()}")
else:
    print("cpu")
    device = torch.device("cpu")

cuda 0


## Load data

Loads all the different datasets: Regions, TS profiles, Bathymetri, GHRSST and SMOS.\
The TS profiles also gets cleaned and split into different regions

In [3]:
(
    regions,
    ts_profiles,
    bathymetri_lat, bathymetri_lon, bathymetri_topography,
    ghrsst_lat, ghrsst_lon, ghrsst_sst, ghrsst_times, ghrsst_fraction_time, ghrsst_time_bnds,
    smos_sss, smos_time, smos_lat, smos_lon
) = dataHandler.load_all(verbose=True)

MissingConfigException: Primary config directory not found.
Check that the config directory '/zhome/ae/4/138518/Desktop/Special_kursus/modules/config' exists and readable

In [ ]:
ts_areas = dataprocessing.split_regions(ts_profiles, regions.set_crs(4326))

## Creation of training, validation and testing datasets

The following code splits the dataset into 3 datasets (training, valdiation and testing).\
The data outside the ghrsst time is cut.

In [ ]:
min_time, max_time = min(ghrsst_fraction_time), max(ghrsst_fraction_time)
training_end = 2012
validation_end = 2015
train, val, test, areas = dataprocessing.split_data_set(ts_areas, training_end, validation_end, min_time, max_time)

# Training

Converts the dict objects into dataloaders containing torch tensors and scales each feature

In [ ]:
scalers = [
    scaling.MeanScaling, # lar
    scaling.MeanScaling, # lon
    scaling.MeanScaling, # year
    scaling.MeanScaling, # decimal year
    scaling.MeanScaling, # sss
    scaling.MeanScaling, # sst
    scaling.MeanScaling, # surface depth
    scaling.MinMaxScaling, # bathymetri
    scaling.MeanScaling, # salinity profile
    scaling.MeanScaling, # temperature profile
]

scalers, train_loader, val_loader, test_loader = dataprocessing.process_data(
    train=train,
    val=val,
    test=test,
    bathymetri_lat=bathymetri_lat,
    bathymetri_lon=bathymetri_lon,
    bathymetri_topography=bathymetri_topography,
    scalers= scalers
)

Trains the model and saves it

In [ ]:
training_loss, validation_loss = models.train_model(train_loader, val_loader)